
# Pandas Brutal Labs — All-in-One (70 Labs + Electrical Madness + Frog Puzzles)

**How to use:** Run the Setup cell first. Each Lab has a short goal and runnable solution. 
The puzzles (Electrical Madness & Frog) are implemented using Pandas operations like `shift`, `merge`, `groupby`.

**Tip:** Prefer vectorized Pandas/Numpy ops over Python loops. Use `astype`, `to_datetime`, `merge`, `groupby`, `agg`, `apply` (sparingly), `explode`, `stack/unstack`, `pivot`, `melt`, and `Categorical` wisely.


## Setup & Utilities

In [ ]:
import pandas as pd
import numpy as np
import re #re: Python’s regex engine—useful for string cleaning and pattern extraction.
import time 
# Measuring durations (time.perf_counter())
# For more precise timing in data workflows, prefer time.perf_counter() over time.time().
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 50)

def seed(n=42):
    np.random.seed(n)
# This is a simple convenience wrapper so seed() sets NumPy’s global RNG state.
# Calling seed(42) means all subsequent np.random.* calls (e.g., randn, permutation, choice) become reproducible.

print('pandas:', pd.__version__)


pandas: 3.0.0



---

### LAB 1: DataFrame from two-dimensional list
**Goals:** Create DF, set columns, dtypes


In [11]:
seed(42) # not much use here
# print(seed.__doc__)
# print(seed(42))
#  this seed(42) is to ensure reproducibility as it sets the random number generator to a fixed state
data = [
    [1, "Alice", 10.5],
    [2, "Bob",   20.0],
    [3, "Cara",  15.2],
]

df = pd.DataFrame(
    data,
    columns=['id','name','score']
).astype({
    'id':    'Int64',   # pandas nullable integer type
    'name':  'string',  # pandas string type
    'score': 'float64'  # standard float type
})

print(df.dtypes)
display(df)


id         Int64
name      string
score    float64
dtype: object


,id,name,score
0,1,Alice,10.5
1,2,Bob,20.0
2,3,Cara,15.2



---

### LAB 2: From dict of arrays/lists
**Goals:** Align lengths, dtype inference


In [ ]:
# << 1 >>
# Pandas builds columns column-wise (not row-wise like in Lab 1).
# Column order follows dict insertion order (Python 3.7+ guarantees it), so the final column order will be: id, name, score.

# << 2 >> 
# You didn’t provide an index ---> Pandas creates a default RangeIndex(0, 3).
# So rows are 0, 1, 2.

# << 3 >> 
# id: [1, 2, 3] ---> all ints ---> inferred as int64 (platform dependent, but typically 64-bit).
# name: ['A', 'B', 'C'] ---> Python strings ---> inferred as object dtype (i.e., references to Python str objects).
# score: np.array([10.0, 20.0, 30.0]) ---> a NumPy array of floats ---> stays float64.


""" << -+---------------------------+- >> """

# Pandas is happy with either a list or a NumPy array; dtype inference would still arrive at float64 here.
# Where it matters:

# Performance/Copy behavior: If you pass a NumPy array, Pandas may avoid copying in some cases (constructor tries to be zero-copy when safe). Lists always need conversion.
# Dtype control: NumPy arrays already have a dtype; Pandas will respect it unless it must cast (e.g., mixing with incompatible values).
# example ==> forcing a specific dtype via NumPy

# np.array([10, 20, 30], dtype=np.float32)  # yields float32
# Pandas will typically keep float32 for the column rather than upcasting to float64

""" << -+---------------------------+- >> """

# --> name becomes object, not “string dtype”

# By default, Pandas uses object for text columns (backed by Python str objects).
# If you need consistent missing semantics (<NA>), better vectorized string ops, and sometimes memory wins, convert to the dedicated string dtype:


# --> Nullable integers vs plain integers

# int64 cannot represent missing values. If later you insert NaN into id, Pandas will upcast the entire column to float64 (e.g., 1 → 1.0).
# If id may have missing values, use nullable Int64 (capital I): df = df.astype({'id': 'Int64'})


""" << -+- ORIGNAL CODE -+- >> """

df = pd.DataFrame({'id':[1,2,3], 'name':['A','B','C'], 'score':np.array([10.0,20.0,30.0])})
print(df)


""" << -+- VARIATIONS -+- >> """

    """ << -+- 1 -+- >> """
# so for a improvemennt we can use id as an index by suign set_index
df = df.set_index('id')
print(df)

    """ << -+- 2 -+- >> """
# Use better text dtype and nullable ints right away
# df = (pd.DataFrame({'id':[1,2,3], 'name':['A','B','C'], 'score':[10.0,20.0,30.0]})
#         .astype({'id': 'Int64', 'name': 'string', 'score': 'float64'}))

# BUT SINCE WE ALREADY MADE DF WE CAN JUST DO
df = df.astype({'id': 'Int64', 'name': 'string', 'score': 'float64'})
print(df)

    """ << -+- 3 -+- >> """
# Using NumPy array with specific dtype
df = pd.DataFrame({
    'id':   np.array([1,2,3], dtype=np.int32),
    'name': np.array(['A','B','C'], dtype='U10'),
    'score':np.array([10.0,20.0,30.0], dtype=np.float32)
})
print(df)



   id name  score
0   1    A   10.0
1   2    B   20.0
2   3    C   30.0
   name  score
id            
1     A   10.0
2     B   20.0
3     C   30.0



---

### LAB 3: From list of lists with columns
**Goals:** Assign column names


In [ ]:

""" << -+- ORIGNAL CODE -+- >> """
rows = [[101,'x'],[102,'y']]
# When to use category
# If dept has low cardinality and repeats a lot (e.g., 'HR', 'ENG', 'FIN'), categoricals save memory and speed up group-bys:
df = pd.DataFrame(rows, columns=['emp_id','dept'])

""" << -+- ITERATIONS -+- >> """
""" << -+- ----------------------------------- -+- >> """
""" << -+- 1 -+- >> """
# USING CATEGORIES IN VALUES THAT HAVE A LOW CARDINALITY
# TERMONLOGY: Cardinality = number of unique values in a column. Low cardinality means few unique values relative to total rows.
# Example: dept with values like 'HR', 'ENG', 'FIN' repeated across many rows has low cardinality, while a column like 'emp_id'
#    with unique values for each employee has high cardinality.
# Categorical dtype is ideal for low-cardinality columns because it stores data as integer codes with a separate mapping to the actual 
#   category values, which can save memory and speed up operations like group-bys and comparisons.

# Categories list (unique values)
# -- ['x', 'y']

# 2. Codes (integer mapping for each row)
# -- 0, 1, 0, 0, 1, 0, 1, ...

# So instead of storing strings repeatedly, Pandas stores:
# -- 2 strings: 'x' and 'y'
# -- an integer array for the data         # Huge efficiency win.

df['dept'] = df['dept'].astype('category')
print(df)

""" << -+- ----------------------------------- -+- >> """

display(df)


IndentationError: unexpected indent (2420110864.py, line 9)


---

### LAB 4: From list of tuples
**Goals:** Tuple rows behave like lists


In [ ]:

rows = [(1,'p'),(2,'q')]
df = pd.DataFrame(rows, columns=['k','v'])
print(df)



---

### LAB 5: From list of dicts
**Goals:** Heterogeneous keys → columns union


In [ ]:

rows = [{'id':1,'name':'A'},{'id':2,'name':'B','city':'NY'}]
df = pd.DataFrame(rows)
print(df)



---

### LAB 6: From nested dict (outer keys → columns or index)
**Goals:** Normalize nested dict to columns


In [ ]:

nested = {'r1':{'a':1,'b':2}, 'r2':{'a':3,'b':4,'c':5}}
df = pd.DataFrame.from_dict(nested, orient='index').reset_index(names='row')
print(df)



---

### LAB 7: From Series
**Goals:** Series → single-column or to_frame


In [ ]:

s = pd.Series([10,20,30], index=['x','y','z'], name='value')
df = s.to_frame()
print(df)



---

### LAB 8: Construct from string data
**Goals:** Convert object→numeric, errors handling


In [ ]:

df = pd.DataFrame({'raw':['  10','20 ',' NotANumber ','30']})
df['raw_clean'] = df['raw'].str.strip()
df['num'] = pd.to_numeric(df['raw_clean'], errors='coerce')
print(df)



---

### LAB 9: Clean string data
**Goals:** strip/lower/replace


In [ ]:

df = pd.DataFrame({'name':['  Alice ','BoB','cARA  ']})
df['norm'] = df['name'].str.strip().str.lower()
df['alpha'] = df['norm'].str.replace(r'[^a-z]','', regex=True)
print(df)



---

### LAB 10: Replace using regex
**Goals:** Regex patterns with `replace`


In [ ]:

df = pd.DataFrame({'txt':['A-123','B_456','C 789']})
df['digits'] = df['txt'].str.replace(r'\D+','', regex=True)
print(df)



---

### LAB 11: Reindexing
**Goals:** Create new index with fill


In [ ]:

df = pd.DataFrame({'v':[1,2,3]}, index=[2,4,6])
print('orig
', df)
print('reindexed
', df.reindex([1,2,3,4,5,6], fill_value=0))



---

### LAB 12: Map external values (Series.map)
**Goals:** Category → code mapping, missing


In [ ]:

df = pd.DataFrame({'tier':['gold','silver','bronze','platinum']})
mp = {'bronze':1,'silver':2,'gold':3}
df['code'] = df['tier'].map(mp)
print(df)



---

### LAB 13: Reset index
**Goals:** `reset_index(drop=True)` vs keep


In [ ]:

df = pd.DataFrame({'a':[1,2,3]}, index=['x','y','z'])
print(df.reset_index())
print(df.reset_index(drop=True))



---

### LAB 14: Rename columns & index
**Goals:** `rename` with dict, `set_axis`


In [ ]:

df = pd.DataFrame({'A':[1,2],'B':[3,4]}, index=['r1','r2'])
print(df.rename(columns={'A':'alpha'}, index={'r1':'row1'}))



---

### LAB 15: Map a column with dict (missing handling)
**Goals:** `map` vs `fillna`


In [ ]:

df = pd.DataFrame({'dept':['ENG','FIN','HR','OPS']})
lookup = {'ENG':'Engineering','FIN':'Finance','HR':'HR'}
df['dept_full'] = df['dept'].map(lookup).fillna('Unknown')
print(df)



---

### LAB 16: map() vs replace()
**Goals:** `map` for transformation; `replace` for direct substitution


In [ ]:

df = pd.DataFrame({'x':['A','B','C','D']})
print('map:')
print(df['x'].map({'A':1,'B':2}))
print('replace:')
print(df['x'].replace({'A':1,'B':2}))



---

### LAB 17: DataFrame.map / applymap fallback
**Goals:** Elementwise function over entire DF


In [ ]:

df = pd.DataFrame([[1,2],[3,4]], columns=['a','b'])
# Pandas provides DataFrame.applymap; DataFrame.map may exist in newer versions. Use applymap here.
df2 = df.applymap(lambda v: v*v)
print(df2)



---

### LAB 18: from_dict & to_dict
**Goals:** records/index/list/orient


In [ ]:

rows = [{'id':1,'name':'A'},{'id':2,'name':'B'}]
df = pd.DataFrame.from_dict(rows)
print('records -> df:
', df)
print('to_dict records:
', df.to_dict(orient='records'))
print('to_dict index:
', df.set_index('id').to_dict(orient='index'))



---

### LAB 19: Multicolumn mapping
**Goals:** map two columns using two lookups


In [ ]:

df = pd.DataFrame({'dept':['ENG','FIN','ENG'], 'level':[1,2,2]})
mp_dept={'ENG':'Engineering','FIN':'Finance'}
mp_level={1:'L1',2:'L2'}
df['dept_full'] = df['dept'].map(mp_dept)
df['level_str'] = df['level'].map(mp_level)
print(df)



---

### LAB 20: Nested dictionaries mapping
**Goals:** map (dept,level) → title using tuple key


In [ ]:

df = pd.DataFrame({'dept':['ENG','ENG','FIN'], 'level':[1,2,2]})
mp = {('ENG',1):'SWE I', ('ENG',2):'SWE II', ('FIN',2):'Analyst II'}
df['title'] = list(map(lambda t: mp.get((t[0], t[1]), 'NA'), df[['dept','level']].itertuples(index=False, name=None)))
print(df)



---

### LAB 21: Iterate over rows (iterrows)
**Goals:** Education-only; avoid for perf


In [ ]:

df = pd.DataFrame({'x':[1,2],'y':[3,4]})
for i, row in df.iterrows():
    print(i, row['x'] + row['y'])



---

### LAB 22: Row iteration with itertuples
**Goals:** Faster than iterrows


In [ ]:

df = pd.DataFrame({'x':[1,2],'y':[3,4]})
for r in df.itertuples(index=True, name='Row'):
    print(r.Index, r.x + r.y)



---

### LAB 23: Selecting rows by conditions
**Goals:** Boolean masks


In [ ]:

df = pd.DataFrame({'name':['Ann','Ben','Cara','Dee'], 'age':[17,20,21,18]})
print(df[df['age']>=18])



---

### LAB 24: Select any row using iloc[] and iat[]
**Goals:** Position-based access


In [ ]:

df = pd.DataFrame({'a':[10,20,30]})
print(df.iloc[1])
print(df.iat[2,0])



---

### LAB 25: Limited rows with given columns
**Goals:** Slice rows/cols


In [ ]:

df = pd.DataFrame({'a':[1,2,3],'b':[4,5,6],'c':[7,8,9]})
print(df.loc[0:1, ['a','c']])



---

### LAB 26: Drop rows on condition
**Goals:** `drop` with mask index


In [ ]:

df = pd.DataFrame({'a':[1,2,3,4]})
df = df.drop(df[df['a']%2==0].index)
print(df)



---

### LAB 27: Insert row at position
**Goals:** Use concat; reindex


In [ ]:

df = pd.DataFrame({'a':[1,3,4]})
row = pd.DataFrame({'a':[2]})
df2 = pd.concat([df.iloc[:1], row, df.iloc[1:]], ignore_index=True)
print(df2)



---

### LAB 28: Create list from rows
**Goals:** records/tuples


In [ ]:

df = pd.DataFrame({'id':[1,2],'name':['A','B']})
print(df.to_dict('records'))
print(list(df.itertuples(index=False, name=None)))



---

### LAB 29: Ranking rows
**Goals:** `rank` methods


In [ ]:

df = pd.DataFrame({'name':['A','B','C','D'],'score':[90,95,90,80]})
df['rank_dense'] = df['score'].rank(method='dense', ascending=False).astype(int)
print(df.sort_values('rank_dense'))



---

### LAB 30: Sorting rows
**Goals:** Single/multi column sort


In [ ]:

df = pd.DataFrame({'a':[2,1,2],'b':[3,2,1]})
print(df.sort_values(['a','b'], ascending=[True, False]))



---

### LAB 31: Row with max/min value
**Goals:** idxmax/idxmin


In [ ]:

df = pd.DataFrame({'x':[5,9,3],'y':[7,1,8]})
print('row of max x:
', df.loc[[df['x'].idxmax()]])
print('row of min y:
', df.loc[[df['y'].idxmin()]])



---

### LAB 32: Rows containing substring
**Goals:** `str.contains`


In [ ]:

df = pd.DataFrame({'text':['foo bar','lorem','barista']})
print(df[df['text'].str.contains('bar')])



---

### LAB 33: Convert column to index
**Goals:** `set_index`


In [ ]:

df = pd.DataFrame({'id':[1,2],'val':[10,20]})
print(df.set_index('id'))



---

### LAB 34: Randomly select rows
**Goals:** `sample`


In [ ]:

df = pd.DataFrame({'a':range(10)})
print(df.sample(3, random_state=42))



---

### LAB 35: Create column (vectorized instead of for-loop)
**Goals:** Avoid Python loops; use vectorization


In [ ]:

df = pd.DataFrame({'a':[1,2,3],'b':[10,20,30]})
# Vectorized:
df['c'] = df['a']*df['b']
print(df)



---

### LAB 36: Get column names
**Goals:** `df.columns`, list


In [ ]:

df = pd.DataFrame({'x':[1],'y':[2]})
print(list(df.columns))



---

### LAB 37: Rename columns
**Goals:** `rename` or `set_axis`


In [ ]:

df = pd.DataFrame({'A':[1],'B':[2]})
print(df.rename(columns={'A':'alpha'}))



---

### LAB 38: Unique values from column
**Goals:** `unique`, `nunique`


In [ ]:

df = pd.DataFrame({'c':['a','b','a','c']})
print(df['c'].unique())
print(df['c'].nunique())



---

### LAB 39: Conditional operation columns
**Goals:** `np.where`


In [ ]:

df = pd.DataFrame({'x':[1,2,3,4]})
df['parity'] = np.where(df['x']%2==0, 'even', 'odd')
print(df)



---

### LAB 40: Return index labels where condition
**Goals:** `index[mask]`


In [ ]:

df = pd.DataFrame({'x':[5,1,5,2]})
mask = df['x']==5
print(df.index[mask].tolist())



---

### LAB 41: Formatting integer column
**Goals:** string formatting


In [ ]:

df = pd.DataFrame({'acct':[1,23,456]})
df['acct_str'] = df['acct'].map(lambda v: f"ACC-{v:04d}")
print(df)



---

### LAB 42: New column from existing
**Goals:** combine columns


In [ ]:

df = pd.DataFrame({'first':['A','B'],'last':['x','y']})
df['user'] = df['first'].str.lower() + '_' + df['last']
print(df)



---

### LAB 43: Column from condition
**Goals:** chain conditions


In [ ]:

df = pd.DataFrame({'score':[35,60,85]})
conds = [df['score']<50, df['score'].between(50,79), df['score']>=80]
vals = ['fail','pass','distinction']
df['grade'] = np.select(conds, vals, default='NA')
print(df)



---

### LAB 44: Split string into columns (regex)
**Goals:** `str.extract` / `str.split(expand=True)`


In [ ]:

df = pd.DataFrame({'raw':['A-12','B_34','C 56']})
print(df['raw'].str.extract(r'(?P<letter>[A-Z])\D*(?P<num>\d+)'))



---

### LAB 45: Frequency counts
**Goals:** `value_counts`


In [ ]:

df = pd.DataFrame({'c':['a','b','a','c','a']})
print(df['c'].value_counts())



---

### LAB 46: Split a text column into two
**Goals:** `str.split(expand=True, n=1)`


In [ ]:

df = pd.DataFrame({'name':['Last, First','Doe, Jane']})
last_first = df['name'].str.split(',', n=1, expand=True)
last_first.columns=['last','first']
print(last_first)



---

### LAB 47: Index of min value
**Goals:** `idxmin`


In [ ]:

df = pd.DataFrame({'v':[5,3,7,1,9]})
print(df['v'].idxmin())



---

### LAB 48: Index of max value
**Goals:** `idxmax`


In [ ]:

df = pd.DataFrame({'v':[5,3,7,1,9]})
print(df['v'].idxmax())



---

### LAB 49: Difference of two columns
**Goals:** Vectorized subtraction


In [ ]:

df = pd.DataFrame({'a':[10,20],'b':[3,8]})
df['diff'] = df['a'] - df['b']
print(df)



---

### LAB 50: n-largest values from a column
**Goals:** `nlargest`


In [ ]:

df = pd.DataFrame({'x':[5,1,9,7,3]})
print(df['x'].nlargest(3))



---

### LAB 51: n-smallest values from a column
**Goals:** `nsmallest`


In [ ]:

df = pd.DataFrame({'x':[5,1,9,7,3]})
print(df['x'].nsmallest(2))



---

### LAB 52: Drop one/multiple columns
**Goals:** `drop(columns=...)`


In [ ]:

df = pd.DataFrame({'a':[1],'b':[2],'c':[3]})
print(df.drop(columns=['b','c']))



---

### LAB 53: Lowercase column names
**Goals:** rename with comprehension


In [ ]:

df = pd.DataFrame({'First Name':[1], 'LAST_NAME':[2]})
df.columns = [c.lower() for c in df.columns]
print(df)



---

### LAB 54: Capitalize first letter of a column
**Goals:** `str.title` or custom


In [ ]:

df = pd.DataFrame({'city':['new york','san francisco']})
print(df['city'].str.title())



---

### LAB 55: Uppercase a column
**Goals:** `str.upper`


In [ ]:

df = pd.DataFrame({'code':['ab','Cd']})
print(df['code'].str.upper())



---

### LAB 56: Basics of Time Series Manipulation
**Goals:** `to_datetime`, set index, sort_index


In [ ]:

df = pd.DataFrame({'ts':['2023-01-01','2023-01-03','2023-01-02'], 'v':[1,3,2]})
df['ts'] = pd.to_datetime(df['ts'])
df = df.set_index('ts').sort_index()
print(df)



---

### LAB 57: Timedelta & Period indexes
**Goals:** `pd.timedelta_range`, `pd.period_range`


In [ ]:

print(pd.timedelta_range('0 days', periods=3, freq='2H'))
print(pd.period_range('2024-01', periods=3, freq='M'))



---

### LAB 58: Convert string column → datetime
**Goals:** `to_datetime(errors='coerce')`


In [ ]:

df = pd.DataFrame({'when':['2024/01/01','bad','2024-03-05']})
df['ts'] = pd.to_datetime(df['when'], errors='coerce')
print(df)



---

### LAB 59: Electrical Madness I — 2D Potential via DataFrame.shift
**Goals:** Stencil Jacobi update using df.shift


In [ ]:

N=16
V = pd.DataFrame(np.zeros((N,N)))
V.iloc[0,:] = 1.0   # top 1V, others 0V
for _ in range(200):
    avg = (V.shift(1, axis=0).fillna(0) + V.shift(-1, axis=0).fillna(0) +
           V.shift(1, axis=1).fillna(0) + V.shift(-1, axis=1).fillna(0)) / 4.0
    V.iloc[1:-1,1:-1] = avg.iloc[1:-1,1:-1]
    V.iloc[0,:] = 1.0; V.iloc[-1,:]=0.0; V.iloc[:,0]=0.0; V.iloc[:,-1]=0.0
print(round(float(V.iloc[N//2, N//2]), 4))



---

### LAB 60: Electrical Madness II — Discrete Laplacian in Pandas
**Goals:** Use shifts to compute Laplacian residual


In [ ]:

N=16
V = pd.DataFrame(np.zeros((N,N)))
V.iloc[0,:] = 1.0
for _ in range(300):
    up=V.shift(1,0).fillna(0); dn=V.shift(-1,0).fillna(0)
    lf=V.shift(1,1).fillna(0); rt=V.shift(-1,1).fillna(0)
    R = (up+dn+lf+rt - 4*V)
    V = V + 0.25*R
    V.iloc[0,:] = 1.0; V.iloc[-1,:]=0.0; V.iloc[:,0]=0.0; V.iloc[:,-1]=0.0
print(V.iloc[:3,:3])



---

### LAB 61: Electrical Madness III — Graph Laplacian with DataFrames
**Goals:** Edges DataFrame → Laplacian via groupby


In [ ]:

# Build 3x3 grid nodes and edges; compute degree and Laplacian matrix using groupby
n=3; m=3
nodes = [(i*m+j) for i in range(n) for j in range(m)]
edges=[]
for i in range(n):
    for j in range(m):
        u=i*m+j
        for di,dj in [(1,0),(0,1)]:
            ii, jj = i+di, j+dj
            if 0<=ii<n and 0<=jj<m:
                v=ii*m+jj; edges.append((u,v))
E = pd.DataFrame(edges, columns=['u','v'])
# Undirected add reverse
E = pd.concat([E, E.rename(columns={'u':'v','v':'u'})], ignore_index=True)
# Degree
deg = E.groupby('u').size().rename('deg')
L = pd.DataFrame(0, index=nodes, columns=nodes)
for _,row in E.iterrows():
    L.loc[row.u, row.v] -= 1
for k,d in deg.items():
    L.loc[k,k] = d
print(L)



---

### LAB 62: Frog I — 2D Knight Moves via merge
**Goals:** Frontier positions as DataFrame; next via vectorized merge


In [ ]:

N=8
moves = pd.DataFrame([(-2,-1),(-2,1),(-1,-2),(-1,2),(1,-2),(1,2),(2,-1),(2,1)], columns=['dr','dc'])
front = pd.DataFrame({'r':[0],'c':[0]})
seen = front.copy()
for step in range(1,8):
    cand = front.assign(key=1).merge(moves.assign(key=1), on='key').drop('key',axis=1)
    cand['r']=cand['r']+cand['dr']; cand['c']=cand['c']+cand['dc']
    cand = cand[(cand['r'].between(0,N-1)) & (cand['c'].between(0,N-1))]
    front = cand[['r','c']].drop_duplicates().merge(seen, on=['r','c'], how='left', indicator=True)
    front = front[front['_merge']=='left_only'][['r','c']]
    seen = pd.concat([seen, front]).drop_duplicates()
print('reachable cells:', len(seen))



---

### LAB 63: Frog II — 3D Levels + Teleports
**Goals:** Use DataFrames for (level,row,col) transitions


In [ ]:

L,R,C = 3,6,6
front = pd.DataFrame({'l':[0],'r':[0],'c':[0]})
seen = front.copy()
steps2d = pd.DataFrame([(-1,0),(1,0),(0,-1),(0,1)], columns=['dr','dc'])
tele = pd.DataFrame([(1,0,0),(-1,0,0)], columns=['dl','dr','dc'])
for step in range(1,10):
    cand2d = front.assign(key=1).merge(steps2d.assign(key=1), on='key').drop('key',1)
    cand2d['l']=front['l'].values.repeat(len(steps2d))
    cand2d['r']=cand2d['r']+cand2d['dr']; cand2d['c']=cand2d['c']+cand2d['dc']; cand2d['dl']=0
    candt = front.assign(key=1).merge(tele.assign(key=1), on='key').drop('key',1)
    candt['l']=candt['l']+candt['dl']; candt['r']=candt['r']+candt['dr']; candt['c']=candt['c']+candt['dc']
    cand = pd.concat([cand2d[['l','r','c']], candt[['l','r','c']]])
    cand = cand[cand['l'].between(0,L-1) & cand['r'].between(0,R-1) & cand['c'].between(0,C-1)]
    front = cand.drop_duplicates().merge(seen, on=['l','r','c'], how='left', indicator=True)
    front = front[front['_merge']=='left_only'][['l','r','c']]
    seen = pd.concat([seen, front]).drop_duplicates()
print('reachable states:', len(seen))



---

### LAB 64: Second Highest Salary
**Goals:** Drop duplicates then nlargest(2).tail(1)


In [ ]:

df = pd.DataFrame({'salary':[100,200,200,300,150]})
unique = df['salary'].drop_duplicates().nlargest(2)
second = unique.iloc[-1] if len(unique)>=2 else None
print(second)



---

### LAB 65: Rank Scores
**Goals:** dense rank descending


In [ ]:

df = pd.DataFrame({'name':['A','B','C','D'],'score':[90,95,90,80]})
df['rank'] = df['score'].rank(method='dense', ascending=False).astype(int)
print(df.sort_values(['rank','name']))



---

### LAB 66: Invalid Comments (regex)
**Goals:** Keep only alnum & space; flag invalid


In [ ]:

df = pd.DataFrame({'c':['ok comment','bad@@@','another#no']})
df['valid'] = ~df['c'].str.contains(r'[^\w\s]', regex=True)
print(df)



---

### LAB 67: Salesperson Without Orders to Company (anti-join)
**Goals:** left_anti merge pattern


In [ ]:

sales = pd.DataFrame({'sid':[1,2,3],'cust':['A','B','C']})
orders = pd.DataFrame({'sid':[1,1,3],'company':['CorpX','CorpY','CorpX']})
# Want salespeople with no orders to CorpX
corpX = orders[orders['company']=='CorpX'][['sid']].drop_duplicates()
anti = sales.merge(corpX, on='sid', how='left', indicator=True)
print(anti[anti['_merge']=='left_only'])



---

### LAB 68: Unique Projects by Employee
**Goals:** groupby agg(set) + explode


In [ ]:

df = pd.DataFrame({'emp':[1,1,2,2,2],'proj':['P1','P1','P2','P1','P3']})
agg = df.groupby('emp')['proj'].unique().rename('projects').reset_index()
print(agg)



---

### LAB 69: Salary Trends Across Departments
**Goals:** groupby over time; rolling mean


In [ ]:

seed(42)
dates = pd.date_range('2023-01-01', periods=6, freq='M')
df = pd.DataFrame({'date':np.repeat(dates, 2), 'dept':['A','B']*6, 'sal':np.random.randint(80,121, size=12)})
df = df.sort_values('date')
trend = df.groupby('dept').apply(lambda g: g.set_index('date')['sal'].rolling(3, min_periods=1).mean()).rename('sal_ma3').reset_index()
print(trend.head())



---

### LAB 70: Delete Duplicate Phone Numbers
**Goals:** `drop_duplicates` on column


In [ ]:

df = pd.DataFrame({'phone':['111','222','111','333']})
print(df.drop_duplicates('phone'))



---

## Mock Interview (60 minutes)

**1) What are the main differences between `merge`, `join`, and `concat`?**  
*Answer:* `merge` performs SQL-style joins on columns or index; `join` is a convenience for index-based joins (or column-on-index); `concat` stacks objects along an axis (row-wise or column-wise), aligning by index/columns.

**2) When should you prefer `map`/`replace`/`merge` for lookups?**  
*Answer:* Small Series mapping → `map`; simple substitution → `replace`; relational lookup with many-to-one/one-to-many and extra columns → `merge`.

**3) Explain `groupby(...).agg` vs `transform`.**  
*Answer:* `agg` collapses groups to a reduced result; `transform` returns an output aligned with the original index, broadcasting per-group results back (e.g., z-scores, group-means subtraction).

**4) How to detect and handle missing values robustly?**  
*Answer:* Use `isna`/`notna`; choose strategies contextually (`fillna`, `dropna`, impute by group medians), preserve dtypes with `convert_dtypes`.

**5) Vectorization vs `apply` vs Python loops?**  
*Answer:* Prefer vectorized ops; `apply` row-wise is Python-level and slow; use `numexpr`, category types, and `merge`/`groupby` to avoid row loops.

**6) Pitfalls with chained indexing?**  
*Answer:* `df[df.x>0]['y']=...` can set on a copy. Prefer `.loc[mask, 'y']=...` to avoid `SettingWithCopyWarning`.

**7) Wide↔Long reshaping difference between `pivot` and `melt`?**  
*Answer:* `pivot` turns long → wide using index/columns/values, assumes uniqueness; `melt` collapses wide → long.

**8) Time-series resample vs groupby?**  
*Answer:* `resample` requires a DatetimeIndex and bins by time frequency; `groupby(pd.Grouper(freq=...))` can group by time-like keys on columns or index.

**9) How to get second highest per group?**  
*Answer:* Use `drop_duplicates`, `nlargest`, then `groupby('group')['value'].nlargest(2).groupby(level=0).nth(1)`, or sort and `cumcount`.

**10) Memory optimizations?**  
*Answer:* Use `Categorical` for low-cardinality strings, downcast numerics (`to_numeric(..., downcast='integer')`), read CSV with dtype dicts, chunked processing, and `usecols`.
